In [0]:
from google.colab import drive


In [2]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
import os
import cv2
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from PIL import Image,ImageFile
import shutil
import csv
import sys
from random import sample
import numpy as np
#import pandas as pd
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from distutils.dir_util import copy_tree
a = copy_tree("/content/gdrive/My Drive/total_data", "/content/DATA")

In [5]:
print( len([name for name in os.listdir('/content/DATA') ]) )

1037


In [0]:
datasetDir = '/content/DATA'
saveDir = '/content/cropped_images'
resizeDir = '/content/resized_images'

In [7]:
exts = {'jpg'}

for filename in os.listdir(datasetDir):
  s = filename.split('.')
  exts.add(s[1])
exts.remove('csv')
exts.remove('xml')

print(exts)

{'gif', 'jpeg', 'jpg', 'png', 'PNG'}


In [8]:
#code to remove .xml files
'''
for filename in os.listdir(datasetDir):
  if filename.endswith('xml'):
    os.remove(os.path.join(datasetDir,filename))
'''


"\nfor filename in os.listdir(datasetDir):\n  if filename.endswith('xml'):\n    os.remove(os.path.join(datasetDir,filename))\n"

In [9]:
if not os.path.isdir(saveDir):
  os.mkdir(saveDir)

count = 20
boxes = 0
fileno = 1
labelData = [['filename' , 'class']]

for filename in os.listdir(datasetDir):
  if not filename.endswith('.xml'): 
    continue
  
  # if count == 0 : break
  # count = count - 1
  
  
  xname = os.path.join(datasetDir, filename)
  
  e = 'noImg'
  for ext in exts:
    if os.path.isfile(os.path.join(datasetDir,filename.replace('xml',ext)) ):
      e = ext
      break
      
  if(e == 'noImg'):
    print(filename , ' image not found');
    continue
    
  iname = os.path.join(datasetDir, filename.replace('xml',e))
  

  im = Image.open(r''+iname) 
  # print(os.path.isfile(iname) , iname , im.mode)
  if im.mode in ("RGBA", "P"):
    im = im.convert("RGB")
  im.show()

  
  
  tree = ET.parse(xname)
  root = tree.getroot()
#   print(ET.tostring(root, encoding='utf8', method='xml'))
  for elem in root.findall('.//object'):

    name = str(elem.find('name').text)
    
    
    boxes+=1
    b = elem.find('bndbox')
    xmin = int(b.find('xmin').text)
    ymin = int(b.find('ymin').text)
    xmax = int(b.find('xmax').text)
    ymax = int(b.find('ymax').text)
    
    
    im1 = im.crop((xmin,ymin,xmax,ymax))
    
    sDir = os.path.join(saveDir, str(fileno)+'.jpg' )
    labelData.append([str(fileno) , name])
    fileno += 1
    
    im1.save(sDir)

print(boxes)
with open('/content/labels.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(labelData)
csvFile.close()

M 10 (1).xml  image not found
M (9) (1).xml  image not found
1179


In [0]:
#!zip -r /content/file.zip /content/cropped_images

In [11]:
#code to download files
'''
from google.colab import files
files.download('/content/file.zip')
'''

"\nfrom google.colab import files\nfiles.download('/content/file.zip')\n"

In [0]:
if os.path.isdir(resizeDir):
  shutil.rmtree(resizeDir)

In [0]:
if not os.path.isdir(resizeDir):
  os.mkdir(resizeDir)

In [0]:
for item in os.listdir(saveDir):
  if os.path.isfile(os.path.join(saveDir,item)):
    im = Image.open(os.path.join(saveDir,item))
    
    imResize = im.resize((224,224), Image.ANTIALIAS)
    imResize.save(os.path.join(resizeDir,item))

In [0]:
#!zip -r /content/file.zip /content/resized_images

In [16]:
labelData = [x for x in labelData if x not in sample([a for a in labelData if a[1]=='H'], 480) ]
print('Healthy :' , len([a for a in labelData if a[1]=='H']) ) 
print('M :' , len([a for a in labelData if a[1]=='M']))
print('Y :' , len([a for a in labelData if a[1]=='Y']))

Healthy : 301
M : 192
Y : 228


## MODEL : 

In [0]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = 224

In [18]:
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

58892288/58889256 [==============================] - 0s 0us/step


In [19]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f458c156fd0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f450d302080> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f450d302a20> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f45004f2a20> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f45004f28d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f4500503c18> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f450050c358> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f4500516be0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f4500516f60> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f450051b518> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f4500522438> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 

In [20]:
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               12845568  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1539      
Total params: 27,561,795
Trainable params: 19,926,531
Non-trainable params: 7,635,264
_________________________________________________________________


In [0]:
train_data_dir = '/content/train_data'
test_data_dir = '/content/test_data'

In [22]:
print('Data size :' , len(os.listdir(resizeDir)))

Data size : 1179


In [23]:
labelData.remove(['filename', 'class'])
labelData[0]

['1', 'M']

In [24]:
X = [l[0] for l in labelData]
y = [l[1] for l in labelData]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

print('train size:',len(X_train))
print('test size:',len(X_test))


if not os.path.isdir(train_data_dir):
  os.mkdir(train_data_dir)
else:
  shutil.rmtree(train_data_dir)
  os.mkdir(train_data_dir)

for [f,c] in [list(a) for a in zip(X_train , y_train)]:
  if not os.path.isdir(os.path.join(train_data_dir,c)):
    os.mkdir(os.path.join(train_data_dir,c))
  shutil.copy(os.path.join(resizeDir,f+'.jpg') , os.path.join(train_data_dir,c,f+'.jpg'))

if not os.path.isdir(test_data_dir):
  os.mkdir(test_data_dir)
else:
  shutil.rmtree(test_data_dir)
  os.mkdir(test_data_dir)
for [f,c] in [list(a) for a in zip(X_test , y_test)]:
  if not os.path.isdir(os.path.join(test_data_dir,c)):
    os.mkdir(os.path.join(test_data_dir,c))
  shutil.copy(os.path.join(resizeDir,f+'.jpg') , os.path.join(test_data_dir,c,f+'.jpg'))
# with open('/content/train.csv', 'w') as csvFile:
#     writer = csv.writer(csvFile)
#     writer.writerows(labelData)
# csvFile.close()

train size: 483
test size: 238


In [0]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=180,
      width_shift_range=0.2,
      height_shift_range=0.2, 
      horizontal_flip=True,
      fill_mode='nearest')

In [0]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_batchsize = 30
test_batchsize = 10

In [28]:
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_size, img_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_size, img_size),
        batch_size=test_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 483 images belonging to 3 classes.
Found 238 images belonging to 3 classes.


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [30]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=150,
      validation_data=test_generator,
      validation_steps=test_generator.samples/test_generator.batch_size,
      verbose=1)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/150
17/16 [===============================] - 7s 400ms/step - loss: 1.5999 - acc: 0.3892 - val_loss: 1.0609 - val_acc: 0.4538
Epoch 2/150
17/16 [===============================] - 6s 358ms/step - loss: 1.0421 - acc: 0.4803 - val_loss: 1.1569 - val_acc: 0.3824
Epoch 3/150
17/16 [===============================] - 6s 360ms/step - loss: 0.9401 - acc: 0.5093 - val_loss: 0.8642 - val_acc: 0.6218
Epoch 4/150
17/16 [===============================] - 6s 359ms/step - loss: 0.9465 - acc: 0.5818 - val_loss: 1.1150 - val_acc: 0.3529
Epoch 5/150
17/16 [===============================] - 6s 363ms/step - loss: 0.9227 - acc: 0.5714 - val_loss: 0.9130 - val_acc: 0.5504
Epoch 6/150
17/16 [===============================] - 6s 363ms/step - loss: 0.8072 - acc: 0.6315 - val_loss: 0.9131 - val_acc: 0.5588
Epoch 7/150
17/16 [===============================] - 6s 364ms/step - loss: 0.8041 - acc: 0.6522 - val_loss: 0.9090 - va

In [0]:
model.save('deepnail.h5')

In [0]:
model.save_weights('deepnail_weights.h5')

In [0]:
!mv '/content/deepnail.h5' '/content/gdrive/My Drive'